In [4]:
import time
import pandas as pd
from datetime import date
import win32com.client as win32
import os
start_time = time.time()

## Parte 1: Ingreso nombre archivos

In [5]:
# Recurso + OnHand
r_oh = '2023-09-15 ON HAND AVANCE_PG.xlsx'

# 9030 Descarga
descarga_9030 = '9030-DESCARGA-15-09-2023.xlsx'

## Parte 2: Carga archivos + Modificaciones especificas

In [6]:
# 1. Carga Recurso + OnHand
df1 = pd.read_excel(r_oh, sheet_name = 'BASE 1', dtype = 'string', usecols = 'A:S')

# 1.1 Al cargar la data, python lee las comas como puntos, por lo tanto aplicamos la funcion replace para mantenerlas como ','
df1['Ctd.'] = df1['Ctd.'].str.replace(".",",", regex = False)

# Agregar Columnas TU
df1['TU'] = pd.NA
df1['FCH. Creacion TU'] = pd.NA
df1['CENTRO. SUMNISTRO'] = pd.NA

In [7]:
# 2. Carga 9030-DESCARGA dia Actual
df2 = pd.read_excel(descarga_9030, dtype = 'string',
                    usecols = ['Documento','Número de posición','Orden de servicio','Operación de orden de servicio',
                              'UMP.SUP','UMP','Prod.','Descripción de producto','Ctd.','Propietario del stock',
                              'Status de entrada en stock','Status de la entrada de mercancías',
                               'Persona autorizada a disponer', 'TU', 'FCH. Creacion TU', 'CENTRO. SUMNISTRO'])

# 2.1 Renombrar las filas del 9030-DESCARGA. 
df2.rename(columns = {'Orden de servicio':'OS', 'Operación de orden de servicio':'OP', 'UMP.SUP': 'UMp sup.', 
            'UMP':'Unidad manipulación','Propietario del stock': 'Propietario', 'Status de entrada en stock': 'Fecha EM',
          'Status de la entrada de mercancías':'Hora EM','Prod.':'Producto'}, inplace = True)

# 2.2 Creamos las columnas CONCAT y D+I, al igual que el archivo OnHand Avance.
df2['CONCAT'] = df2['Unidad manipulación'] + df2['Producto'] + df2['Ctd.']
df2['D+I'] = df2['Documento'] + df2['Número de posición']

# 2.3 Agregar columnas en archivo 9030-DESCARGA, para así poder realizar concatenado con Recurso+OnHand
df2['Revi'], df2['TP.UBI'], df2['OS+OP+MAT+CDT'], df2['OS+OP+MAT'], df2['OS.SUP'], df2['OSO'], df2['TIPO DE PROCESO'], \
df2['Tipo almacén'], df2['Ubicación'], df2['UMp superior'], df2['Tipo de documento'] = ['', pd.NA, '', '', '', '', 'DESCARGA', 
                                                                                        '9030', '9030-DESCARGA', '','PDI']

# 2.4 Reordenamos las columnas para que sean igual al orden de Recurso+OnHand
df2 = df2.reindex(columns = df1.columns.to_list())

In [8]:
# 3. Arreglo Formato Fecha para que quede como texto

# 3.1 Eliminar '00:00:00'
df2['Fecha EM'] = df2['Fecha EM'].str.replace(" 00:00:00","", regex = False)

# 3.2 Crear Columna Auxiliar
df2['Fecha EM_Aux'] = df2['Fecha EM']

# 3.3 Fecha EM a formato a Datetime
df2['Fecha EM'] = pd.to_datetime(df2['Fecha EM'], errors = 'coerce', format = "%Y/%m/%d", yearfirst = True)

# 3.4 Fecha EM a String
df2['Fecha EM'] = df2['Fecha EM'].dt.strftime("%d/%m/%Y")

# 3.5 Aquellos que no tenian formato fecha , los igualamos a la columna Auxiliar para recuperar dichos valores
df2.loc[pd.isna(df2['Fecha EM']) == True, 'Fecha EM'] = df2['Fecha EM_Aux'].iloc[df2.loc[pd.isna(df2['Fecha EM'])].index.values]

# 3.6 Borrar Fecha Auxiliar
df2.drop(labels = 'Fecha EM_Aux', axis = 1, inplace = True)

C:\Users\pablo.gutierrez\AppData\Local\Temp\ipykernel_7440\28481621.py:16: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df2.loc[pd.isna(df2['Fecha EM']) == True, 'Fecha EM'] = df2['Fecha EM_Aux'].iloc[df2.loc[pd.isna(df2['Fecha EM'])].index.values]


In [9]:
# 4. Cargar BASE 2 On Hand Avance (Toda la hoja)
df_base2 = pd.read_excel(r_oh, sheet_name = 'BASE 2', dtype = 'string')
df4 = df_base2.loc[: , ['UBICACIÓN','OBSERVACION']]

## Parte 3: Modificaciones

In [10]:
# 5. Concatenar

# 5.1 Unir OHA_PG y 9030D
df3 = pd.concat([df1,df2], axis = 0)

# 5.2 Reset Index y eliminar columna index
df3.reset_index(inplace = True)
df3.drop(columns = 'index', axis = 1, inplace = True)

# 5.3 Agregamos identificador unico
df3['Identificador Unico'] = df3.index.values.tolist()

In [11]:
# 6. Si OS = 'Revisado', Entonces OS = OP = 0
df3.loc[df3['OS'] == 'REVISADO', 'OS'] = '0'
df3.loc[df3['OS'] == 'REVISADO', 'OP'] = '0'

In [12]:
# 7. Aplicamos función TEXTO a la Operación
df3.loc[: , 'OP'] = list(map(lambda x: x.zfill(4), df3['OP']))

C:\Users\pablo.gutierrez\AppData\Local\Temp\ipykernel_7440\4266819642.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df3.loc[: , 'OP'] = list(map(lambda x: x.zfill(4), df3['OP']))


In [13]:
# 8. Creamos las columnas nuevas
df3.loc[pd.isna(df3['Unidad manipulación']), 'Unidad manipulación'] = ''
df3['CONCAT'] = df3['Unidad manipulación'] + df3['Producto'] + df3['Ctd.']
df3['OSO'] = df3['OS'] + df3['OP']
df3['OS.SUP'] = [i[:10] + '00' for i in df3['OS']]
df3['OS+OP+MAT'] = df3['OS'] + df3['OP'] + df3['Producto']
df3['OS+OP+MAT+CDT'] = df3['OS'] + df3['OP'] + df3['Producto'] + df3['Ctd.']

In [14]:
# 9. Si la ubicación contiene Excedente, Faltante o Sobrante, entonces de OSOPMATCTD a OP = 0
df3.loc[df3['Ubicación'].str.contains('EXCEDENTE'), ['OS', 'OP', 'OSO', 'OS.SUP', 'OS+OP+MAT', 'OS+OP+MAT+CDT']] = '0'
df3.loc[df3['Ubicación'].str.contains('FALTANTE'), ['OS', 'OP', 'OSO', 'OS.SUP', 'OS+OP+MAT', 'OS+OP+MAT+CDT']] = '0'
df3.loc[df3['Ubicación'].str.contains('SOBRANTE'), ['OS', 'OP', 'OSO', 'OS.SUP', 'OS+OP+MAT', 'OS+OP+MAT+CDT']] = '0'

In [15]:
# 10. Cruce ELIMINAR OS

# 10.1 Eliminar Duplicados columna Ubicación Base 2
df4.drop_duplicates(subset = 'UBICACIÓN', inplace = True)

# 10.2 Definir Key values para realizar el Join. En este caso, el valor común entre ambas columnas es la Ubicación
df3.set_index('Ubicación', inplace = True)
df4.set_index('UBICACIÓN', inplace = True)

# 10.3 Join (BUSCARV en excel)
df5 = df3.join(df4, how = 'left')

# 10.4 Ordenamos la data según la columna 'Identificador Unico' que creamos en un principio
df5.sort_values(by = 'Identificador Unico', inplace = True)

# 10.5 Reset index
df5.reset_index(inplace = True)

# 10.6 Columna Ubicación a su posicion Original
index = df5.pop('index')
df5.insert(loc = 12, column = 'Ubicación', value = index.values)

In [16]:
# 11. Cruce con Base 2 para dejar OSOPMAT = 0 aquellos que nos arroje el

# 11.1 Rellenar con NO ELIMINAR aquellas celdas NA del Join
df5['OBSERVACION'].fillna('NO ELIMINAR', inplace = True)

# 11.2 Si OBSERVACION = ElIMINAR OS, OS+OP+MAT = 0
df5.loc[df5['OBSERVACION'] == 'ELIMINAR OS', 'OS+OP+MAT'] = '0'

# 11.3 Eliminamos la columna Observación
df5.drop(columns = 'OBSERVACION', axis = 1, inplace = True)

In [17]:
# 12. Para todo 'OS' = 0
df5.loc[df5['OS'] == '0' , ['OP', 'OSO', 'OS.SUP', 'OS+OP+MAT', 'OS+OP+MAT+CDT']] = '0'

In [18]:
# 13. Formato Correcto Columna Cantidad
df5['Ctd.'] = df5['Ctd.'].str.replace(",",".", regex = False)
df5['Ctd.'] = df5['Ctd.'].astype('float')

In [19]:
# 14. Ordenamos la data según la columna 'Identificador Unico' que creamos en un principio
df5.sort_values(by = 'Identificador Unico', inplace = True)

In [20]:
# 15. Rellenar Columna TP.UBI 

# 15.1 TP UBI 8034
df5.loc[df5['Tipo almacén'] == '8032', 'TP.UBI'] = '8034'
df5.loc[df5['Tipo almacén'] == '8034', 'TP.UBI'] = '8034'
df5.loc[df5['Ubicación'].str.contains('9020-CRC'), 'TP.UBI'] = '8034'
df5.loc[df5['Ubicación'].str.contains('9020-REGUL-CRC'), 'TP.UBI'] = '8034'
df5.loc[df5['Ubicación'].str.contains('9020-SUBKIT'), 'TP.UBI'] = '8034'

# 15.2 TP.UBI OTROS
df5.loc[(pd.isna(df5['TP.UBI']) == True), 'TP.UBI'] = 'OTROS'

In [21]:
# Orden final de columnas
df5 = df5[['CONCAT','TP.UBI','OS+OP+MAT+CDT','OS+OP+MAT','OS.SUP','OSO','OS','OP','TIPO DE PROCESO',
        'Tipo almacén','Ubicación','UMp sup.','UMp superior','Unidad manipulación','Documento','Número de posición',
        'Tipo de documento','Producto','Descripción de producto','Ctd.','Propietario','Persona autorizada a disponer',
        'Fecha EM','Hora EM','Tipo de stocks','TU','FCH. Creacion TU','CENTRO. SUMNISTRO']]

## Formato Excel

In [22]:
# Espacio entre tablas
df_base2.rename(columns = {'Unnamed: 2':'', 'Unnamed: 6':''}, inplace = True)

# Actualizar Nombre Archivo
today = date.today()
fecha_hoy = today.strftime("%Y-%m-%d")
Nombre_Archivo = fecha_hoy + ' ON HAND AVANCE.xlsx'

# Descargar Archivo
with pd.ExcelWriter(Nombre_Archivo) as writer:  
    df5.to_excel(writer, sheet_name='BASE 1', index = False)
    df_base2.to_excel(writer, sheet_name='BASE 2', index = False)
    
# Abrir Instancia Excel
xlApp = win32.Dispatch('Excel.Application')
xlApp.Visible = True

# Abrir On Hand Avance (Libro)
path = os.path.abspath(Nombre_Archivo)
wb = xlApp.Workbooks.Open(path)

# Ajustar Columnas
wb.Worksheets(1).Activate()
wb.ActiveSheet.Columns.AutoFit()

wb.Worksheets(2).Activate()
wb.ActiveSheet.Columns.AutoFit()

# Eliminar Columnas Inncesarias BASE 1
wb.Worksheets(1).Activate()
wb.ActiveSheet.Columns('A:D').Delete()

# Guardar como XLSB
wb.SaveAs(Filename = os.path.dirname(path) + '\\' + fecha_hoy + ' ON HAND AVANCE', FileFormat = 50)

## Busqueda OS para obtener Activación y QPLUSS ECC

In [23]:
# 18.1 Insertar Columna Plan
df5.insert(column = 'Plan', loc = 26, value = pd.NA)

# 18.2 Crear Columnas auxiliares 4 primeros digitos y 6 primeros digitos
df5['OS.SUP_AUX'] = list(map(lambda x: x[:4], df5['OS.SUP']))
df5['OS.SUP_KIT'] = list(map(lambda x: x[:6], df5['OS.SUP']))

# 18.3 Replacements
df5.loc[df5['OS.SUP_AUX'] == '5028', 'Plan'] = 'PLAN CSAR'
df5.loc[df5['OS.SUP_AUX'] == '5082', 'Plan'] = 'PLAN CRE'
df5.loc[df5['OS.SUP_AUX'] == '5059', 'Plan'] = 'PLAN CRC'
df5.loc[df5['OS.SUP_KIT'] == '000001', 'Plan'] = 'PLAN KIT'
df5.loc[(pd.isna(df5['Plan'])) == True , 'Plan'] = pd.NA

# 18.4 Eliminar Columnas Auxiliares
df5.drop(labels = ['OS.SUP_AUX', 'OS.SUP_KIT'], axis = 1, inplace = True)

# 18.5 Columna OS solo con los Clientes de Interés
busqueda_os = df5.loc[pd.isna(df5['Plan']) == False, 'OS']
busqueda_os = pd.DataFrame({'QPLUSS ECC' : busqueda_os.values})

# 18.6 ELiminar Duplicados
busqueda_os.drop_duplicates(keep = 'first', inplace = True)

# 18.7 OS.SUP para busqueda ACTIVACIÓN
busqueda_os['ACTIVACIÓN'] = [i[:10] + '*' for i in busqueda_os['QPLUSS ECC']]

# 18.8 Descargar Archivo
busqueda_os.to_excel('Busqueda_Activación_QPLUSSECC.xlsx', index = False)

In [24]:
# 17. Calculo Tiempo Procesamiento
end_time = time.time()
total_time = end_time - start_time
total_time = total_time / 60
total_time = "{:.2f}".format(total_time)
total_time = str(total_time)
print('Tiempo Total: ' + total_time + ' minutos')

Tiempo Total: 2.60 minutos
